##imports

In [ ]:
"""
Источник: https://github.com/NVIDIA/NeMo/blob/main/tutorials/asr/Online_Noise_Augmentation.ipynb
"""

#Установка необходимых пакетов
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]
!pip install librosa==0.9.2
!pip install json
!pip install soundfile==0.12.1

In [ ]:
import json
from nemo.collections.asr.parts.preprocessing import perturb, segment
import glob
import soundfile as sf
import os
import librosa

* Перейдите в раздел "Общий доступ" в Google Диске.
* Выберите SPbU_smart-assistant
* Щелкните ее правой кнопкой мыши и выберите Добавить ярлык на диск.
* Появится всплывающее окно, выберите MyDrive и нажмите Добавить ярлык.
* Теперь перейдите в блокнот Google Colab и подключитесь к Google Диску с помощью кнопки


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [ ]:
!git clone https://github.com/spbu-smart-assistant/support-chatbot.git

##train test split based on an existing tsv file

In [12]:
rows = [i*15 for i in range(16)]

In [53]:
table = pd.read_csv('/content/support-chatbot/data/raw/Transcriptions.tsv', sep='\t')
crop = table.iloc[rows, :2]
crop

,Tags,Names
0,"Пароль, Почта, Факультет, Студенческий, Деканат",Audio1_speed_0.wav
15,"Кабинет, Компьютер, Оборудование",Audio2_speed_0.wav
30,"Интернет, Общежитие",Audio3_speed_0.wav
45,"Интернет, Общежитие, Компьютер, Сеть, Оплата",Audio4_speed_0.wav
60,"Блэкборд, Технические работы",Audio5_speed_0.wav
75,"Облачное хранилище, Ст",Audio6_speed_0.wav
90,"Тимс, Пароль, Студенческий, Ст, Почта",Audio7_speed_0.wav
105,"Вайфай, Виндовс, Сеть, Пароль, Студенческий, П...",Audio8_speed_0.wav
120,"Устройства, Печать, Принтер, Библиотека",Audio9_speed_0.wav
135,"Библиотека, Пароль, Студенческий, Почта",Audio10_speed_0.wav


In [57]:
lib = defaultdict(list)
for index, row in crop.iterrows():
  row['Names'] = row['Names'].split('_')[0]
  tags = row['Tags'].replace(" ", '').split(',')
  for tag in tags:
    lib[tag].append(row['Names'])
lib

defaultdict(list,
            {'Пароль': ['Audio1', 'Audio7', 'Audio8', 'Audio10'],
             'Почта': ['Audio1', 'Audio7', 'Audio10', 'Audio14', 'Audio15'],
             'Факультет': ['Audio1'],
             'Студенческий': ['Audio1', 'Audio7', 'Audio8', 'Audio10'],
             'Деканат': ['Audio1'],
             'Кабинет': ['Audio2'],
             'Компьютер': ['Audio2', 'Audio4', 'Audio12'],
             'Оборудование': ['Audio2'],
             'Интернет': ['Audio3', 'Audio4', 'Audio11'],
             'Общежитие': ['Audio3', 'Audio4'],
             'Сеть': ['Audio4', 'Audio8'],
             'Оплата': ['Audio4'],
             'Блэкборд': ['Audio5'],
             'Техническиеработы': ['Audio5'],
             'Облачноехранилище': ['Audio6'],
             'Ст': ['Audio6', 'Audio7'],
             'Тимс': ['Audio7', 'Audio11', 'Audio12', 'Audio13', 'Audio15'],
             'Вайфай': ['Audio8'],
             'Виндовс': ['Audio8', 'Audio16'],
             'ПМПУ': ['Audio8'],
           

In [58]:
remain = set(crop['Names'].values.tolist())

In [56]:
temp = lib.copy()
for key, ar in temp.items():
  if len(ar) < 2:
    lib.pop(key)

In [17]:
srt = sorted(lib.items(), key=lambda kv: len(kv[1]))
srt

[('Общежитие', ['Audio3', 'Audio4']),
 ('Сеть', ['Audio4', 'Audio8']),
 ('Ст', ['Audio6', 'Audio7']),
 ('Виндовс', ['Audio8', 'Audio16']),
 ('Библиотека', ['Audio9', 'Audio10']),
 ('Экзамен', ['Audio14', 'Audio15']),
 ('Компьютер', ['Audio2', 'Audio4', 'Audio12']),
 ('Интернет', ['Audio3', 'Audio4', 'Audio11']),
 ('Устройства', ['Audio9', 'Audio12', 'Audio13']),
 ('Пароль', ['Audio1', 'Audio7', 'Audio8', 'Audio10']),
 ('Студенческий', ['Audio1', 'Audio7', 'Audio8', 'Audio10']),
 ('Почта', ['Audio1', 'Audio7', 'Audio10', 'Audio14', 'Audio15']),
 ('Тимс', ['Audio7', 'Audio11', 'Audio12', 'Audio13', 'Audio15'])]

In [18]:
x, y = [], []

In [19]:
seen = set()
for tag, names in srt:
  for name in names:
    if name not in seen:
      seen.add(name)
      x.append(name)
      y.append(tag)

In [ ]:
x

In [ ]:
remain -= set(x)
remain

In [ ]:
y

In [ ]:
#stratify only if a lot of values
train_ratio = 0.7
test_ratio = 0.3

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1 - train_ratio)#stratify=y

x_train.extend(remain)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
#x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) #stratify=y_test

datas = {
    'train': x_train,
    'test': x_test
    #'val': x_val
}
datas

## TSV 

In [ ]:
temp = [x + '_' for x in datas['train']]
temp

In [ ]:
zxc = table[table['Names'].str.contains('|'.join(temp))]
zxc

In [71]:
zxc.to_csv('train.tsv', sep='\t', index=False)
!cp /content/train.tsv /content/drive/MyDrive/SPbU_smart-assistant

In [ ]:
temp = [x + '_' for x in datas['test']]
temp

In [ ]:
zxc = table[table['Names'].str.contains('|'.join(temp))]
zxc

In [77]:
zxc.to_csv('test.tsv', sep='\t', index=False)
!cp /content/test.tsv /content/drive/MyDrive/SPbU_smart-assistant

##augs

In [26]:
print("******")
if not os.path.exists("./raw"):
  os.makedirs("./raw")
  for key in datas.keys():
    os.makedirs(f"./raw/{key}")

******


In [27]:
for key, vals in datas.items():
  for name in vals:
    audio_path = f"/content/drive/MyDrive/SPbU_smart-assistant/raw/{name}.wav"
    sample, sr = librosa.core.load(audio_path)
    name = audio_path.split('/')[-1]
    sf.write(f'./raw/{key}/{name}', sample, samplerate=sr)

# for audio_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/raw/*.*"):
#   sample, sr = librosa.core.load(audio_path)
#   name = audio_path.split('/')[-1]
#   sf.write(f'./raw/{name}', sample, samplerate=sr)
print("Конец.\n******")

Конец.
******


In [28]:
#Вспомогательная функция из Nemo
def load_audio(filepath) -> segment.AudioSegment:
    sample_segment = segment.AudioSegment.from_file(filepath, target_sr=sr)
    return sample_segment

In [29]:
#Функция создания манифеста шумов
def write_manifest(filepath, data_dir='.', manifest_name='noise_manifest', duration_max=1e9, duration_stride=1.0):
              
    with open(os.path.join(data_dir, manifest_name + '.json'), 'w') as fout:
        y, sr = librosa.load(filepath)
        duration = librosa.get_duration(y=y, sr=sr)

        offsets = []
        durations = []

        if duration > duration_max:
            current_offset = 0.0

            while current_offset < duration:
                difference = duration - current_offset
                segment_duration = min(duration_max, difference)

                offsets.append(current_offset)
                durations.append(segment_duration)

                current_offset += duration_stride

        else:
            offsets.append(0.0)
            durations.append(duration)


        for duration, offset in zip(durations, offsets):
            metadata = {
                'audio_filepath': filepath,
                'duration': duration,
                'label': 'noise',
                'text': '_',  # for compatibility with ASRAudioText collection
                'offset': offset,
            }

            json.dump(metadata, fout)
            fout.write('\n')
            fout.flush()

        name = noise_path.split('/')[-1]
        print(f"Wrote {len(durations)} segments for filename {name}")
            
    print("Finished preparing manifest !")

In [30]:
if not os.path.exists("./noises"):
  os.makedirs("./noises")

for noise_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/noises/*.*"):
  sample, sr = librosa.core.load(noise_path)
  name = noise_path.split('/')[-1]
  sf.write(f'./noises/{name}', sample, samplerate=sr)

In [ ]:
#Манифесты для всех аудио шумов
if not os.path.exists("./manifests"):
  os.makedirs("./manifests")

for noise_path in glob.glob("./noises/*.*"):
  name = noise_path.split('/')[-1][:-4]
  write_manifest(noise_path, data_dir='./manifests', manifest_name=f'noise_{name}', duration_stride=1.0, duration_max=30.0)

In [32]:
import numpy as np
num = 15 #(умножается на количество оригинальных записей)
count = [i for i in range(num)]
first, second, third = np.array_split(count, 3)

In [ ]:
#Здесь используется speedperturburation, вероятности шумов снижены
#Вероятности аугументаций
probas = [0.5, 0.5, 0.5, 1.0]

for i in first:
  #пайплайн аугументаций
  perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-80, max_level=-40),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=30),
    #при изменении количества аудио шумов заменить 7 на новое количество
    perturb.NoisePerturbation(manifest_path=f'./manifests/noise_{i%7+1}.json',
                              min_snr_db=20, max_snr_db=10, max_gain_db=30.0),
    perturb.SpeedPerturbation(sr, 'kaiser_best', min_speed_rate=0.5, 
                              max_speed_rate=2.0, num_rates=-1)]

  augmentations = list(zip(probas, perturbations))
  audio_augmentations = perturb.AudioAugmentor(augmentations)
  for key in datas.keys():
    for path in glob.glob(f"./raw/{key}/*.*"):
      sample_segment = load_audio(path)
      audio_augmentations.perturb(sample_segment)
      name = path.split('/')[-1][:-4]


      sf.write(f"./drive/MyDrive/SPbU_smart-assistant/auto/{key}/{name}_speed_{i}.wav", 
              sample_segment.samples, samplerate=sr, subtype='PCM_24')
      print(f"{key} {name}_speed_{i}.wav wrote")

In [ ]:
#Здесь используется TimeStretchPerturbation, вероятности шумов снижены
#Вероятности аугументаций
probas = [0.5, 0.5, 0.5, 1.0]

for i in second:
  #пайплайн аугументаций
  perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-80, max_level=-40),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=30),
    #при изменении количества аудио шумов заменить 7 на новое количество
    perturb.NoisePerturbation(manifest_path=f'./manifests/noise_{i%7+1}.json',
                              min_snr_db=20, max_snr_db=10, max_gain_db=30.0),
    perturb.TimeStretchPerturbation(min_speed_rate=0.5, max_speed_rate=2, 
                                    num_rates=-1)]

  augmentations = list(zip(probas, perturbations))
  audio_augmentations = perturb.AudioAugmentor(augmentations)

  for key in datas.keys():
    for path in glob.glob(f"./raw/{key}/*.*"):
      sample_segment = load_audio(path)
      audio_augmentations.perturb(sample_segment)
      name = path.split('/')[-1][:-4]

      sf.write(f"./drive/MyDrive/SPbU_smart-assistant/auto/{key}/{name}_stretch_{i}.wav", 
              sample_segment.samples, samplerate=sr, subtype='PCM_24')
      print(f"{key} {name}_stretch_{i}.wav wrote")

In [ ]:
#Здесь не используются временные аугументации, вероятности шумов 1
#Вероятности аугументаций
probas = [1.0, 1.0, 1.0]

for i in third:
  #пайплайн аугументаций
  perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-60, max_level=-35),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=30),
    #при изменении количества аудио шумов заменить 7 на новое количество
    perturb.NoisePerturbation(manifest_path=f'./manifests/noise_{i%7+1}.json',
                              min_snr_db=20, max_snr_db=10, max_gain_db=30.0)]

  augmentations = list(zip(probas, perturbations))
  audio_augmentations = perturb.AudioAugmentor(augmentations)

  for key in datas.keys():
    for path in glob.glob(f"./raw/{key}/*.*"):
      sample_segment = load_audio(path)
      audio_augmentations.perturb(sample_segment)
      name = path.split('/')[-1][:-4]

      sf.write(f"./drive/MyDrive/SPbU_smart-assistant/auto/{key}/{name}_noise_{i}.wav", 
              sample_segment.samples, samplerate=sr, subtype='PCM_24')
      print(f"{key} {name}_noise_{i}.wav wrote")